# Llama 3.2 1B
- all modules and utils are left in nb for educational purpose
- model params (change from Llama 3.1 8B):
    - vocab size 128256 -> 128256
    - input embedding dim 4096 -> 2048
    - context length 8192 -> 131_000
    - masked grouped-query attention w/ 32 heads -> 32 heads
    - n_layers of the the attention 32 -> 16
    - RoPE -> RoPE scaling
    - final feedfordward layer: Swish + SwiGLU+Linear as gate, hidden layer dim 14336 -> 8192
    - add back weight tying:
        reuses / shares weights between token -> input embedding layer and final feedfordward output layer
- NOTE: 
    - GPT applies the positional embeddings to the inputs
    - Llama applies rotations to the query and key vectors in the self-attention mechanism itself

In [ ]:
import torch

In [ ]:
import numpy as np
import os
import sys 
import math
from typing import Tuple, Dict, List

cwd = os.getcwd()
base_path = cwd[:-13]

In [ ]:
print(torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import torch.nn.functional as F

In [ ]:
LLAMA32_1B_CONFIG = {
    "vocab_size": 128_256,      # Vocabulary size
    "context_length": 131_072,  # Context length
    "emb_dim": 2048,            # NEW: Half the embedding dimension
    "n_heads": 32,              # Number of attention heads
    "n_layers": 16,             # NEW: Half the number of layers
    "hidden_dim": 8192,         # NEW: Almost half the size of the intermediate dimension in FeedForward
    "n_kv_groups": 8,           # Key-Value groups for grouped-query attention
    "rope_base": 50_000,        # The base in RoPE's "theta"
    "dtype": torch.bfloat16,    # Lower-precision dtype to save memory
    "rope_freq": {              # RoPE frequency scaling
        "factor": 32.0,         # NEW: Adjustment of the rescaling factor
        "low_freq_factor": 1.0,
        "high_freq_factor": 4.0,
        "original_context_length": 8192,
    }
}

LLAMA32_3B_CONFIG = {
    "vocab_size": 128_256,      # Vocabulary size
    "context_length": 131_000,  # Context length
    "emb_dim": 3072,            # Embedding dimension
    "n_heads": 24,              # Number of attention heads
    "n_layers": 28,             # Number of layers
    "hidden_dim": 8192,         # Size of the intermediate dimension in FeedForward
    "n_kv_groups": 8,           # Key-Value groups for grouped-query attention
    "rope_base": 50_000,        # The base in RoPE's "theta"
    "dtype": torch.bfloat16,    # Lower-precision dtype to save memory
    "rope_freq": {              # RoPE frequency scaling
        "factor": 32.0,
        "low_freq_factor": 1.0,
        "high_freq_factor": 4.0,
        "original_context_length": 8192,
    }
}

In [ ]:
# file name to import functions from 
from helper import import_defs_from_notebook
fullname = "converting-gpt-to-llama2"
names = ["precompute_rope_params", "compute_rope", "SiLU", "FeedForward", "RMSNorm", "MultiHeadAttention"]

imported_module = import_defs_from_notebook(fullname, names)

In [ ]:
# We need to redefine precompute_rope_params
# precompute_rope_params = getattr(imported_module, "precompute_rope_params", None)
compute_rope = getattr(imported_module, "compute_rope", None)
SiLU = getattr(imported_module, "SiLU", None)
FeedForward = getattr(imported_module, "FeedForward", None)
RMSNorm = getattr(imported_module, "RMSNorm", None)

# MultiHeadAttention only for comparison purposes
MultiHeadAttention = getattr(imported_module, "MultiHeadAttention", None)

In [ ]:
def tie_weights(model):
    """
    Ties the weights of the input embedding and output linear layers
    ...
        self.out_layer = nn.Linear(in_features=cfg["emb_dim"],
                                    out_features=cfg["vocab_size"],
                                    bias=False,
                                    dtype=cfg["dtype"])
    ...
        x = self.final_norm(x)
        logits = self.out_layer(x.to(torch.bfloat16))
    
    """
    model.lm_head.weight = model.embed_tokens.weight

